In [1]:
import pandas as pd
import numpy as np
import multiprocessing

from fbprophet import Prophet
from datetime import datetime
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.metrics import mean_absolute_error as mae
from joblib import Parallel, delayed
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import numpy as np
import scipy.stats
import plotly.express as px

In [2]:
def Var_Opt(features, model, X_train, y_train): # function used to create parellel jobs 
    model.fit(X_train[features], y_train)
    return {'features': features, 
            'train score': np.round(model.score(X_train[features], y_train),3),
            'CV score': np.round(model.best_score_, 3)}

In [3]:
def lagger(x_vec, y_vec, x_lag, y_forecast): # function to lag features correctly and consistently
    y_vec = pd.Series(y_vec)
    x_vec = pd.Series(x_vec)
    
    best_corr = []  
    for i in range(x_lag+1):
        best_corr.append({'lag': i,
                          'corr': np.abs((y_vec.shift(-y_forecast).corr(x_vec.shift(i))))})
    return x_vec.shift(pd.DataFrame(best_corr).sort_values('corr')[-1:].lag.values[0])

In [4]:
def preprocess_data(data, x_lag, y_forecast, remove_rows): # preprocess datat
    
    res = {}
    for state in data.state.unique():
        if state in ['MP',       # remove islands
                     'AS',
                     'GU',
                     'PR', 
                     'VI']:
            continue
        print('State: ', state)
        tmp_state = data[data.state == state] # combine data to state level and aggregation
        tmp = []
        for date in tmp_state.date.unique():
            tmp_state_day = tmp_state[tmp_state.date == date]
            tmp.append({'total_deceased': sum(tmp_state_day.total_deceased),
                        'average_temperature': np.mean(tmp_state_day.average_temperature),
                        'minimum_temperature': np.mean(tmp_state_day.minimum_temperature),
                        'maximum_temperature': np.mean(tmp_state_day.maximum_temperature),
                        'rainfall': np.mean(tmp_state_day.rainfall),
                        'dew_point': np.mean(tmp_state_day.dew_point),
                        'relative_humidity': np.mean(tmp_state_day.relative_humidity),
                        'google': np.mean(tmp_state_day.google),
                        'new_confirmed': sum(tmp_state_day.new_confirmed)})
        tmp = pd.DataFrame(tmp)
        
        for col in tmp.columns:
            if col != 'date':
                tmp[col+'_lag'] = lagger(tmp[col], tmp['new_confirmed'], x_lag, y_forecast)
        
        tmp = tmp.iloc[remove_rows:]
        tmp.reset_index(drop=True, inplace = True)
        
        res[state] = {'All Data':tmp, 
                      'X': tmp.iloc[:-y_forecast],
                      'X_future': tmp.iloc[-y_forecast:],
                      'y': tmp['new_confirmed'].shift(-y_forecast).iloc[:-y_forecast],
                     }
        
    return(res)

In [5]:
start_date_back = 182  # pre prepreprocessing to include google data 
end_date_back = 2
x_lag = 14
y_forecast = 7
remove_rows = 30

df1 = pd.read_csv("https://storage.googleapis.com/covid19-open-data/v2/main.csv", low_memory=False)
df1 = df1.dropna(subset=['subregion1_code'])
df1 = df1[df1.country_code == 'US']
df1 = df1[['date',
           'subregion1_code',
           'total_deceased',
           'average_temperature', 
           'minimum_temperature',
           'maximum_temperature', 
           'rainfall', 
           'dew_point', 
           'relative_humidity',
           'new_confirmed']]
df1.columns = ['date',
               'state',
               'total_deceased',
               'average_temperature', 
               'minimum_temperature',
               'maximum_temperature', 
               'rainfall', 
               'dew_point', 
               'relative_humidity',
               'new_confirmed']
df1['days_back'] = [(datetime.now() - day).days for day in pd.to_datetime(df1.date)]
df1 = df1[df1['days_back']<=start_date_back]
df1 = df1[df1['days_back']>=end_date_back]
df1.fillna(0, inplace = True)

df2 =  pd.read_csv("./google_clean copy.csv")
df2 =  pd.read_csv("./google_clean copy.csv")
df2.columns = ['date', 
               'state', 
               'direction', 
               'lag', 
               'google']
df2 = df2[['date','state','google']]

df3 = pd.merge(df1, df2,  how='inner', on=['date', 'state'])
df3.sort_values(['days_back'], inplace = True, ascending = False)
df3.reset_index(drop = True, inplace = True)

In [6]:
data = preprocess_data(df3, x_lag, y_forecast, remove_rows)

State:  AK
State:  VA
State:  IL
State:  IN
State:  ID
State:  IA
State:  KY
State:  LA
State:  OR
State:  MD
State:  MA
State:  ME
State:  KS
State:  UT
State:  CA
State:  WV
State:  CT
State:  CO
State:  FL
State:  DE
State:  DC
State:  WY
State:  AL
State:  AR
State:  AZ
State:  GA
State:  WA
State:  HI
State:  VT
State:  WI
State:  SD
State:  NE
State:  SC
State:  NJ
State:  NH
State:  TN
State:  NC
State:  ND
State:  PA
State:  OK
State:  OH
State:  NY
State:  NV
State:  NM
State:  RI
State:  MO
State:  TX
State:  MI
State:  MN
State:  MT
State:  MS


In [7]:
for state in data.keys(): # attain best features for all states
    rf = RandomForestRegressor()
    rf.fit(data[state]['X'], data[state]['y'])
    feature_importance = pd.DataFrame()
    feature_importance['feature'] = data[state]['X'].columns
    feature_importance['importance'] = rf.feature_importances_
    feature_importance.sort_values(['importance'], ascending=False, inplace=True)
    feature_importance.reset_index(drop = True, inplace = True)
    data[state]['Feature Importance'] = feature_importance[feature_importance['importance']>0]

In [8]:
pipe = Pipeline(steps=[('estimator', SVR())])  #pipeline to utilize all models --stackoverflow.com
params_grid = [{
                'estimator':[SVR()],
                'estimator__C': [1, 10, 100, 1000],
                'estimator__gamma': [0.001, 0.0001],
                },
                {
                'estimator': [RandomForestRegressor()],
                'estimator__n_estimators': [25, 50, 100, 150, 200],
                'estimator__max_depth': [1,2,3,4,5],
                },
    
               {'estimator':[LinearRegression()],
                'estimator__fit_intercept':[True, False]},
    
               {'estimator':[ElasticNet(max_iter=2000)],
                'estimator__alpha':[0, .1, .2, .5, .7, .9, 1],
                'estimator__l1_ratio':[0, .1, .2, .5, .7, .9, 1]},
    
                {'estimator':[KNeighborsRegressor()],
                'estimator__n_neighbors':[3, 5, 7, 9],
                'estimator__weights':['uniform', 'distance']}
              ]




In [9]:
num_cores = multiprocessing.cpu_count() # parallel processing for modeling AWS
for state in np.sort(list(data.keys())):
    print(state)
    gridCV = GridSearchCV(pipe, params_grid, scoring = 'neg_mean_absolute_error')
    features_list = tqdm([list(data[state]['Feature Importance'].feature[:i+1].values) for i in range(len(data[state]['Feature Importance']))])
    processed_list = Parallel(n_jobs=num_cores)(delayed(Var_Opt)(features, 
                                                                 gridCV, 
                                                                 data[state]['X'], 
                                                                 data[state]['y']) for features in features_list)
    best_features = pd.DataFrame(processed_list)
    best_features.sort_values('CV score', ascending = False, inplace = True)
    best_features.reset_index(drop=True, inplace=True)
    data[state]['Best Features'] = best_features



  0%|          | 0/18 [00:00<?, ?it/s]

AK


100%|██████████| 18/18 [00:00<00:00, 4367.04it/s]

AL



100%|██████████| 18/18 [00:00<00:00, 13059.59it/s]

...

WI



100%|██████████| 18/18 [00:00<00:00, 5374.25it/s]

WV



100%|██████████| 18/18 [00:00<00:00, 10039.56it/s]

WY


In [24]:
best_features

,features,train score,CV score
0,[total_deceased_lag],-32.650,-38.739
1,"[total_deceased_lag, total_deceased, new_confi...",-36.505,-44.275
2,"[total_deceased_lag, total_deceased, new_confi...",-36.523,-44.521
3,"[total_deceased_lag, total_deceased, new_confi...",-36.538,-44.565
4,"[total_deceased_lag, total_deceased, new_confi...",-36.545,-44.819
5,"[total_deceased_lag, total_deceased, new_confi...",-36.536,-44.848
6,"[total_deceased_lag, total_deceased, new_confi...",-36.517,-44.903
7,"[total_deceased_lag, total_deceased, new_confi...",-36.696,-46.031
8,"[total_deceased_lag, total_deceased, new_confi...",-36.978,-46.073
9,"[total_deceased_lag, total_deceased, new_confi...",-37.591,-46.122


In [10]:
state_cv_scores = pd.DataFrame([{'State':state, 'CV Score':data[state]['Best Features']['CV score'].iloc[0]} for state in data.keys()])
state_cv_scores.sort_values(['CV Score'], inplace = True)


In [11]:
future_pred_res = [] # Predicting future confirmed cases by state
for state in data.keys():
    top_feat = data[state]['Best Features']['features'].iloc[0]
    X_train = data[state]['X'][top_feat]
    y = data[state]['y']
    X_future = data[state]['X_future'][top_feat]

    future_pred_res.append({'State':state, 
                            'Average Future Prediction '+str(y_forecast)+' Days':np.mean(gridCV.fit(X_train,y).predict(X_future))})

/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 207816.48859370395, tolerance: 131.52554433962266
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegres

/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 128997.66307360555, tolerance: 181.17835849056604
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 171448.83581758183, tolerance: 181.73478691588787
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 114498.2010175118, tolerance: 53.706461682242995
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/

/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 207869.89402488785, tolerance: 131.52554433962266
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 154994.0203628286, tolerance: 175.77894433962265
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 129043.73399667918, tolerance: 181.17835849056604
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/

/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6887277.505222643, tolerance: 3438.2300311926606
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the num

/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 189643.91477560438, tolerance: 3873.489667889908
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 821504.5751775205, tolerance: 3438.2300311926606
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 333638.3665068988, tolerance: 3922.9449027522937
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_c

...

/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1071011.7570324577, tolerance: 1789.9265212962964
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegres

/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 769913.7092393063, tolerance: 2095.093543518519
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 348054.27661796944, tolerance: 198.3777435185185
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9918.439886707114, tolerance: 2076.819351851852
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coo

/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 16471187.128683029, tolerance: 5566.269396296298
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/pipeline.py:335: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegress

/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13055070.650797857, tolerance: 4296.270100000001
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20163691.200349785, tolerance: 6571.649718518518
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20456604.90778958, tolerance: 6712.876666666668
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_co

/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 16604980.338069014, tolerance: 5566.269396296298
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13241415.499271832, tolerance: 4296.270100000001
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20311533.027973127, tolerance: 6571.649718518518
  model = cd_fast.enet_coordinate_descent(
/home/ec2-user/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_c

In [12]:
last_known = [] # getting mean for that last 7 days of data
for state in data.keys():
    last_known.append({'State':state, 
                       'Average Last 7 Days':np.mean(data[state]['y'].iloc[-7:])})


In [13]:
tmp1 = state_cv_scores.merge(pd.DataFrame(future_pred_res))

In [14]:
tmp2 = tmp1.merge(pd.DataFrame(last_known))

In [15]:
tmp2['difference'] = tmp2['Average Future Prediction '+str(y_forecast)+' Days'] - tmp2['Average Last 7 Days']

In [16]:
tmp2['Percent Change'] = np.round(tmp2['difference']/tmp2['Average Last 7 Days']*100)

In [17]:
tmp2.sort_values(['Percent Change'], ascending=False)

,State,CV Score,Average Future Prediction 7 Days,Average Last 7 Days,difference,Percent Change
7,AL,-794.404,6466.399127,3030.857143,3435.541984,113.0
8,WI,-770.649,9082.798701,7049.142857,2033.655844,29.0
5,TN,-904.881,5850.644350,4565.857143,1284.787207,28.0
40,NM,-111.652,1557.030261,1254.571429,302.458832,24.0
41,ND,-98.890,2036.394448,1653.428571,382.965876,23.0
16,PA,-452.375,4016.060643,3370.857143,645.203500,19.0
19,MA,-371.299,1955.333333,1656.857143,298.476190,18.0
11,SC,-591.316,2195.951465,1879.714286,316.237180,17.0
17,OH,-416.979,4841.779354,4211.142857,630.636497,15.0
35,NE,-208.087,2017.146134,1771.714286,245.431849,14.0


In [18]:
tmp2.to_csv('./statepreds.csv')

In [19]:
data['DC']['Feature Importance']

,feature,importance
0,new_confirmed_lag,0.164111
1,new_confirmed,0.139589
2,total_deceased,0.104069
3,total_deceased_lag,0.093687
4,relative_humidity_lag,0.068852
5,maximum_temperature_lag,0.053397
6,relative_humidity,0.045526
7,minimum_temperature,0.043096
8,average_temperature_lag,0.037645
9,dew_point,0.037333


In [41]:
df1 = data['AR']['Feature Importance']

fig = px.bar(df1, x=df1['feature'],y=df1['importance'], color=df1['feature'], title = 'Arkansas Feature Importance')
fig.update_xaxes(title_text='Feature')
fig.update_yaxes(title_text='Feature Importance')
fig.show()